<a href="https://colab.research.google.com/github/a3d2ddk/post-collision-trajectory-estimation-from-impact-pose/blob/main/collision_trajectory_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('Cloning repository...')
!git clone --quiet https://github.com/a3d2ddk/post-collision-trajectory-estimation-from-impact-pose.git
!mv post-collision-trajectory-estimation-from-impact-pose repo
print('Repository cloned!')

Cloning repository...
Repository cloned!


In [2]:
import os, glob
need_pytorch3d=False

try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    os.chdir('/content/repo/misc')

    wheel_files = glob.glob('*.whl')
    wheel = max(wheel_files, key=os.path.getmtime)
    print("Latest created wheel:", wheel)

    # We try to install PyTorch3D from repo wheel.
    !pip install {wheel}
    #!pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

    print('Packages installed!')

Latest created wheel: pytorch3d-0.7.8-cp312-cp312-linux_x86_64.whl
Processing ./pytorch3d-0.7.8-cp312-cp312-linux_x86_64.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=db645cca34e03afe0251be47f17c230b1eb82ec3525b2575258a0a6db52cdefe
  Stored in directory: /root/.cache/pip/wheels/7c/96/04/4f5f31ff812f684f69f40cb1634357812220aac58d4698048c
Successfully built iopath
Packages installed!


In [3]:
import pytorch3d